In [1]:
%%capture
!pip install flickrapi

# Scrape links from flickr

In [2]:
import flickrapi
import requests
import pandas as pd

In [3]:
import flickr_credentials

api_key = flickr_credentials.KEY
api_secret = flickr_credentials.SECRET

In [4]:
URL_FORMAT = 'https://farm{farm_id}.staticflickr.com/{server_id}/{id}_{secret}.jpg'

In [5]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='etree')

In [6]:
photos = flickr.walk(
    tag_mode='all',
    tags='azulejo,tile'
)

links = []

for photo in photos:
    link = URL_FORMAT.format(
        farm_id = photo.get('farm'),
        server_id = photo.get('server'),
        id=photo.get('id'),
        secret=photo.get('secret')
    )
    links.append(link)   

In [7]:
photos = flickr.walk(
    tag_mode='all',
    tags='azulejo'
)

links = []

for photo in photos:
    link = URL_FORMAT.format(
        farm_id = photo.get('farm'),
        server_id = photo.get('server'),
        id=photo.get('id'),
        secret=photo.get('secret')
    )
    links.append(link)   

In [8]:
photos = flickr.walk(
    tag_mode='all',
    tags='tiles,portugal'
)

links = []

for photo in photos:
    link = URL_FORMAT.format(
        farm_id = photo.get('farm'),
        server_id = photo.get('server'),
        id=photo.get('id'),
        secret=photo.get('secret')
    )
    links.append(link)   

In [9]:
pd.Series(links).to_csv('links.csv', index=False, header=False)

# Downloading files locally

In [10]:
import csv
import re
import requests
import logging
import concurrent.futures

In [11]:
logger = logging.getLogger()
logger.setLevel('INFO')

In [12]:
with open('links.csv', 'r') as f:
    reader = csv.reader(f)
    links = [_[0] for _ in list(reader)]

In [13]:
links = links[1:]

In [14]:
def save_link(link):
    name = re.findall('.*/(.*).jpg', link)[0]
    try:
        with open('data/raw_images/{name}.jpg'.format(name=name), 'wb') as f:
            f.write(requests.get(link).content)
        logger.info('Downloaded {link}'.format(link=link))
    except Exception as e:
        logger.error(e)

In [15]:
save_link(links[0])

In [16]:
%%time

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    [
        executor.submit(save_link, link)
        for link in links
    ]

CPU times: user 1min 50s, sys: 16.5 s, total: 2min 7s
Wall time: 1min 19s


In [17]:
len(links)

8196

In [18]:
len(set(links))

3926